# Color Match Challenge:
- Concept: Players are shown a color name and a color swatch. They must decide if the name matches the swatch within a time limit.
- Twist: Add levels where the color names are in different languages or use shades that are very close to each other.

In [1]:
# Import necessary libraries
import pygame
import random
import time
import json

# Initialize Pygame
pygame.init()

# Set up display
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Color Match Challenge")

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# List of colors and their names in different languages
colors = {
    "Red": (255, 0, 0),
    "Green": (0, 255, 0),
    "Blue": (0, 0, 255),
    "Yellow": (255, 255, 0),
    "Purple": (128, 0, 128),
    "Orange": (255, 165, 0),
    "Pink": (255, 192, 203),
    "Brown": (165, 42, 42),
    "Gray": (128, 128, 128),
    "Black": (0, 0, 0)
}

color_names = {
    "English": ["Red", "Green", "Blue", "Yellow", "Purple", "Orange", "Pink", "Brown", "Gray", "Black"],
    "Spanish": ["Rojo", "Verde", "Azul", "Amarillo", "Púrpura", "Naranja", "Rosa", "Marrón", "Gris", "Negro"],
    "French": ["Rouge", "Vert", "Bleu", "Jaune", "Violet", "Orange", "Rose", "Marron", "Gris", "Noir"],
    "German": ["Rot", "Grün", "Blau", "Gelb", "Lila", "Orange", "Rosa", "Braun", "Grau", "Schwarz"]
}

# Mapping of color names in different languages to their English equivalents
color_name_mapping = {
    "Rojo": "Red",
    "Verde": "Green",
    "Azul": "Blue",
    "Amarillo": "Yellow",
    "Púrpura": "Purple",
    "Naranja": "Orange",
    "Rosa": "Pink",
    "Marrón": "Brown",
    "Gris": "Gray",
    "Negro": "Black",
    "Rouge": "Red",
    "Vert": "Green",
    "Bleu": "Blue",
    "Jaune": "Yellow",
    "Violet": "Purple",
    "Orange": "Orange",
    "Rose": "Pink",
    "Marron": "Brown",
    "Gris": "Gray",
    "Noir": "Black",
    "Rot": "Red",
    "Grün": "Green",
    "Blau": "Blue",
    "Gelb": "Yellow",
    "Lila": "Purple",
    "Orange": "Orange",
    "Rosa": "Pink",
    "Braun": "Brown",
    "Grau": "Gray",
    "Schwarz": "Black"
}

# Set up fonts
font = pygame.font.Font(None, 74)
small_font = pygame.font.Font(None, 36)

# Load sound effects
correct_sound = pygame.mixer.Sound('correct.wav')
wrong_sound = pygame.mixer.Sound('wrong.wav')

# Function to display text on the screen
def display_text(text, font, color, x, y):
    text_surface = font.render(text, True, color)
    screen.blit(text_surface, (x, y))

# Function to display color swatch
def display_swatch(color, x, y, width, height):
    pygame.draw.rect(screen, color, (x, y, width, height))

# Function to display the result message
def display_result(message, color):
    display_text(message, small_font, color, screen_width // 2 - 100, screen_height // 1.5)

# Function to display the timer
def display_timer(time_left):
    display_text(f"Time: {time_left}", small_font, BLACK, screen_width - 150, 50)

# Function to display the score
def display_score(score):
    display_text(f"Score: {score}", small_font, BLACK, 50, 50)

# Function to display the high score
def display_high_score(high_score):
    display_text(f"High Score: {high_score}", small_font, BLACK, 50, 100)

# Function to display the game over screen
def game_over_screen(score, high_score):
    screen.fill(WHITE)
    display_text("Game Over", font, BLACK, screen_width // 2 - 150, screen_height // 3)
    display_text(f"Final Score: {score}", small_font, BLACK, screen_width // 2 - 100, screen_height // 2)
    display_text(f"High Score: {high_score}", small_font, BLACK, screen_width // 2 - 100, screen_height // 1.8)
    display_text("Press R to Restart", small_font, BLACK, screen_width // 2 - 100, screen_height // 1.5)
    pygame.display.flip()
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    waiting = False

# Function to save the game state
def save_game_state(high_score):
    with open('game_state.json', 'w') as file:
        json.dump({'high_score': high_score}, file)

# Function to load the game state
def load_game_state():
    try:
        with open('game_state.json', 'r') as file:
            return json.load(file)['high_score']
    except (FileNotFoundError, KeyError):
        return 0

# Function to display the language selection screen
def language_selection_screen():
    screen.fill(WHITE)
    display_text("Select Language", font, BLACK, screen_width // 2 - 200, screen_height // 4)
    languages = list(color_names.keys())
    for i, lang in enumerate(languages):
        display_text(f"{i + 1}. {lang}", small_font, BLACK, screen_width // 2 - 100, screen_height // 2 + i * 50)
    pygame.display.flip()
    
    selected_language = None
    while selected_language is None:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
            if event.type == pygame.KEYDOWN:
                if event.key in [pygame.K_1, pygame.K_2, pygame.K_3, pygame.K_4]:
                    selected_language = languages[event.key - pygame.K_1]
    return selected_language

# Main game loop
def main():
    running = True
    score = 0
    level = 1
    time_limit = 5  # Initial time limit in seconds
    high_score = load_game_state()
    language = language_selection_screen()

    while running:
        start_time = time.time()
        color_name = random.choice(color_names[language])
        color_value = colors[color_name_mapping[color_name]]
        match = random.choice([True, False])
        if not match:
            color_name = random.choice(color_names[language])

        while time.time() - start_time < time_limit:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            screen.fill(WHITE)
            display_text(color_name, font, BLACK, screen_width // 2 - 100, screen_height // 4)
            display_swatch(color_value, screen_width // 2 - 50, screen_height // 2, 100, 100)
            time_left = int(time_limit - (time.time() - start_time))
            display_timer(time_left)
            display_score(score)
            display_high_score(high_score)

            keys = pygame.key.get_pressed()
            if keys[pygame.K_y]:
                if match:
                    score += 1
                    correct_sound.play()
                    display_result("Correct!", (0, 255, 0))
                else:
                    wrong_sound.play()
                    display_result("Wrong!", (255, 0, 0))
                pygame.display.flip()
                time.sleep(1)
                break
            elif keys[pygame.K_n]:
                if not match:
                    score += 1
                    correct_sound.play()
                    display_result("Correct!", (0, 255, 0))
                else:
                    wrong_sound.play()
                    display_result("Wrong!", (255, 0, 0))
                pygame.display.flip()
                time.sleep(1)
                break

            pygame.display.flip()
            time.sleep(0.1)

        level += 1
        time_limit = max(1, time_limit - 0.5)  # Decrease time limit but not below 1 second

        if level > 10:  # End game after 10 levels
            if score > high_score:
                high_score = score
                save_game_state(high_score)
            game_over_screen(score, high_score)
            score = 0
            level = 1
            time_limit = 5
            language = language_selection_screen()

    pygame.quit()

if __name__ == "__main__":
    main()


pygame 2.6.0 (SDL 2.28.4, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: video system not initialized